In [146]:
import os

In [148]:
%pwd

'D:\\Project\\Text-Summarizer-Project'

In [ ]:
os.chdir("../")

In [149]:
%pwd

'D:\\Project\\Text-Summarizer-Project'

In [138]:
from dataclasses import dataclass
from pathlib import Path

#"Entity" is a return type of a function.
@dataclass(frozen=True)
class DataIngestionConfig:
    #it's a data class, mention the variable used in config.yaml for data ingestion.
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [139]:
#read data from config.yaml and params.yaml
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [140]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [141]:
import os
#urllib.request is used to download the dataset from the url
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [142]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [143]:
#Creation of pipline for Data Ingestion
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

ParserError: expected '<document start>', but found '<scalar>'
  in "config\config.yaml", line 1, column 3